In [128]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import scale

print(tf.__version__)

2.1.0


In [129]:
df = pd.read_csv("boston.csv",header=0)
print(df.describe)

<bound method NDFrame.describe of         CRIM   ZN   INDUS   CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0    0.00632  18.0    2.31     0  0.538  6.575  65.2  4.0900    1  296   
1    0.02731   0.0    7.07     0  0.469  6.421  78.9  4.9671    2  242   
2    0.02729   0.0    7.07     0  0.469  7.185  61.1  4.9671    2  242   
3    0.03237   0.0    2.18     0  0.458  6.998  45.8  6.0622    3  222   
4    0.06905   0.0    2.18     0  0.458  7.147  54.2  6.0622    3  222   
..       ...   ...     ...   ...    ...    ...   ...     ...  ...  ...   
501  0.06263   0.0   11.93     0  0.573  6.593  69.1  2.4786    1  273   
502  0.04527   0.0   11.93     0  0.573  6.120  76.7  2.2875    1  273   
503  0.06076   0.0   11.93     0  0.573  6.976  91.0  2.1675    1  273   
504  0.10959   0.0   11.93     0  0.573  6.794  89.3  2.3889    1  273   
505  0.04741   0.0   11.93     0  0.573  6.030  80.8  2.5050    1  273   

     PTRATIO  LSTAT  MEDV  
0       15.3   4.98  24.0  
1       17.8   9.14  

In [130]:
ds = df.values
x_data = ds[:,:12]
y_data = ds[:,12]
for i in range(12):
    x_data[:,i] = (x_data[:,i]-x_data[:,i].min())/(x_data[:,i].max()-x_data[:,i].min())
print('x_data shape=',x_data.shape)
print('y_data shape=',y_data.shape)

x_data shape= (506, 12)
y_data shape= (506,)


In [131]:
train_num = 300
valid_num = 100
test_num = len(x_data)- train_num -valid_num

x_train = x_data[:train_num]
y_train = y_data[:train_num]

x_valid = x_data[train_num:train_num+valid_num]
y_valid = y_data[train_num:train_num+valid_num]

x_test = x_data[train_num+valid_num:train_num+valid_num+test_num]
y_test = y_data[train_num+valid_num:train_num+valid_num+test_num]

x_train = tf.cast(scale(x_train), dtype=tf.float32)
x_valid = tf.cast(scale(x_valid), dtype=tf.float32)
x_test = tf.cast(scale(x_test), dtype=tf.float32)

In [132]:
def model(x, w, b):
    return tf.matmul(x,w)+ b

W = tf.Variable(tf.random.normal([12,1],mean=0.0, stddev = 1.0, dtype=tf.float32))
B = tf.Variable(tf.zeros(1), dtype = tf.float32)

print(W)
print(B)

<tf.Variable 'Variable:0' shape=(12, 1) dtype=float32, numpy=
array([[-0.3708039 ],
       [-0.22504778],
       [ 0.6215024 ],
       [ 0.95534444],
       [-0.83417124],
       [ 0.82997173],
       [-0.10736343],
       [-0.4319501 ],
       [ 0.47323957],
       [-0.8861462 ],
       [ 1.3418446 ],
       [ 1.2536004 ]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>


In [133]:
training_epochs = 200
learning_rate = 0.0005
batch_size = 10
def loss(x,y,w,b):
    err = model(x,w,b)-y
    squared_err = tf.square(err)
    return tf.reduce_mean(squared_err)
def grad(x,y,w,b):
    with tf.GradientTape() as tape:
        loss_ = loss(x,y,w,b)
    return tape.gradient(loss_, [w,b])
optimizer = tf.keras.optimizers.SGD(learning_rate)

In [134]:
loss_list_train = []
loss_list_valid = []
total_step = int (train_num/batch_size)

for epoch in range(training_epochs):
    for step in range(total_step):
        xs = x_train[step*batch_size:(step+1)*batch_size,:]
        ys = y_train[step*batch_size:(step+1)*batch_size]
        
        grads = grad(xs,ys,W,B)
        optimizer.apply_gradients(zip(grads,[W,B]))
        
    loss_train = loss(x_train, y_train,W ,B).numpy()
    loss_valid = loss(x_valid, y_valid,W ,B).numpy()
    loss_list_train.append(loss_train)
    loss_list_valid.append(loss_valid)
    print("epoch={:3d} ,train_loss={:.4f},valid_loss={:.4f}".format(epoch+1,loss_train,loss_valid))

epoch=  1 ,train_loss=695.1601,valid_loss=491.2909
epoch=  2 ,train_loss=659.2318,valid_loss=461.7909
epoch=  3 ,train_loss=625.7086,valid_loss=434.6664
epoch=  4 ,train_loss=594.3671,valid_loss=409.6500
epoch=  5 ,train_loss=565.0190,valid_loss=386.5214
epoch=  6 ,train_loss=537.5034,valid_loss=365.0980
epoch=  7 ,train_loss=511.6807,valid_loss=345.2250
epoch=  8 ,train_loss=487.4287,valid_loss=326.7707
epoch=  9 ,train_loss=464.6390,valid_loss=309.6206
epoch= 10 ,train_loss=443.2138,valid_loss=293.6747
epoch= 11 ,train_loss=423.0650,valid_loss=278.8441
epoch= 12 ,train_loss=404.1119,valid_loss=265.0493
epoch= 13 ,train_loss=386.2802,valid_loss=252.2185
epoch= 14 ,train_loss=369.5016,valid_loss=240.2863
epoch= 15 ,train_loss=353.7126,valid_loss=229.1929
epoch= 16 ,train_loss=338.8538,valid_loss=218.8829
epoch= 17 ,train_loss=324.8700,valid_loss=209.3056
epoch= 18 ,train_loss=311.7094,valid_loss=200.4136
epoch= 19 ,train_loss=299.3234,valid_loss=192.1625
epoch= 20 ,train_loss=287.6666,

epoch=162 ,train_loss=102.3767,valid_loss=129.7453
epoch=163 ,train_loss=102.3812,valid_loss=129.8232
epoch=164 ,train_loss=102.3857,valid_loss=129.8994
epoch=165 ,train_loss=102.3903,valid_loss=129.9741
epoch=166 ,train_loss=102.3951,valid_loss=130.0473
epoch=167 ,train_loss=102.3999,valid_loss=130.1190
epoch=168 ,train_loss=102.4048,valid_loss=130.1892
epoch=169 ,train_loss=102.4098,valid_loss=130.2581
epoch=170 ,train_loss=102.4148,valid_loss=130.3256
epoch=171 ,train_loss=102.4199,valid_loss=130.3917
epoch=172 ,train_loss=102.4251,valid_loss=130.4566
epoch=173 ,train_loss=102.4303,valid_loss=130.5202
epoch=174 ,train_loss=102.4356,valid_loss=130.5826
epoch=175 ,train_loss=102.4409,valid_loss=130.6438
epoch=176 ,train_loss=102.4462,valid_loss=130.7039
epoch=177 ,train_loss=102.4516,valid_loss=130.7628
epoch=178 ,train_loss=102.4571,valid_loss=130.8205
epoch=179 ,train_loss=102.4626,valid_loss=130.8773
epoch=180 ,train_loss=102.4680,valid_loss=130.9329
epoch=181 ,train_loss=102.4736,

In [135]:
print("Test_loss:{:.4f}".format(loss(x_test, y_test, W, B).numpy()))
#print("W:{:.4f}".format(W.numpy()))
#print("B:{:.4f}".format(B))
print(W)
print(B)

Test_loss:136.0787
<tf.Variable 'Variable:0' shape=(12, 1) dtype=float32, numpy=
array([[ 0.29020333],
       [ 0.2885908 ],
       [-0.64415365],
       [ 0.69968086],
       [-2.180317  ],
       [ 1.5975026 ],
       [-0.06919503],
       [-1.2852566 ],
       [ 0.50730675],
       [-0.5218967 ],
       [-2.6796637 ],
       [-0.6442877 ]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([25.431242], dtype=float32)>
